# Tutorial: Evolutionary Model Merging with Mergenetic

In this notebook, we illustrate how to perform evolutionary merging of language models using the Mergenetic library. We go step by step, following the logic of the script you provided. This tutorial is focused on a cross-lingual math merging experiment. 

The notebook covers the following steps:

1. **Data Loading** – Read the CSV dataset.
2. **Anchor Extraction** – Randomly select anchor points.
3. **Pre-Evaluation (Optional)** – Evaluate the base models (can be skipped).
4. **Theta Retrieval** – Obtain the latent ability parameters.
5. **Train/Test Split** – Separate the sampled anchors from the rest of the data.
6. **Set Performance Estimation Parameters** – Unpack and configure the parameters for performance estimation.
7. **Define the Merger** – Create the merging object (using SlerpMerger in this example).
8. **Define the Optimization Problem** – Specify the problem instance that wraps the merger, evaluation data, and settings.
9. **Define the Evolutionary Algorithm** – Set up a genetic algorithm (GA) with sampling, crossover, and mutation operators.
10. **Run the Search** – Execute the evolutionary search and test the merged model.

Let’s begin!

In [ ]:
# Step 0: Import required modules
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM

import argparse
import pandas as pd
import numpy as np
import yaml

from mergenetic.searcher import Searcher
from mergenetic.merging import SlerpMerger
from mergenetic.optimization.predefined_problems import (
    CrossLingualMathProblem,
    ConfigPE,
)
from end2end_utils import evaluate_model, retrieve_thetas, ConfigCrossLingual

# For this tutorial, we assume that the required packages and modules are installed and that the
# 'mergenetic' library along with 'end2end_utils' is available in your PYTHONPATH.

print("All modules imported successfully.")

## Configuration Setup

In a regular script you might use command-line arguments and load a YAML file for configuration. In this tutorial we directly define a sample configuration (an instance of `ConfigCrossLingual`). Adjust the values as needed for your experiment.

In [ ]:
# Step 1: Define a sample configuration
# In practice, you could load this from a YAML file.
config_dict = {
    "dataset_path": "data/cross_lingual_math.csv",  # path to your CSV file
    "n_clusters": 10,
    "seed": 42,
    "pop_size": 25,
    "n_iter": 7,
    "run_id": "cross_lingual_math_experiment",
    "bench": "math",
    "mode": "gmpirt",
    "correctness_metric": "accuracy",
    "ft_model_paths": ["base_model_path", "finetuned_model_path"],
    "path_to_store_yaml": "results/configs/",
    "path_to_store_merged_model": "results/merged_models/",
    "dtype": "fp16",
    "device": "cuda",
    "eval_batch_size": 64,
    "lang_id": "en",
    "no_preeval": False,
    # additional parameters as needed
}

# Create a ConfigCrossLingual object from the dictionary
config = ConfigCrossLingual(**config_dict)

print(f"Configuration: {config}")

## Step 1: Load the Data

We load our dataset using `pandas`. Make sure your CSV file is in the correct path.

In [ ]:
# Load the dataset
df = pd.read_csv(config.dataset_path)
print("STEP 1 completed: Data loaded")
print(f"Data shape: {df.shape}")

## Step 2: Extract Random Anchor Points

We randomly select a set of anchor points from the dataset. These anchors will be used later in the evaluation and performance estimation.

In [ ]:
# Extract random anchor points
anchors = df.sample(n=config.n_clusters, random_state=config.seed).index
anchors_weights = np.ones(len(anchors)) / len(anchors)

print("STEP 2 completed: Anchors extracted")
print(f"Anchors: {anchors.tolist()}")

## Step 3: (Optional) Pre-Evaluation of Base Models

If your configuration requires it (i.e. if `no_preeval` is false), this step evaluates the base models on the dataset. In this example, we call `evaluate_model()`. You can skip this step if the pre-evaluation is not needed.

In [ ]:
if config.no_preeval:
    print("STEP 3: Skipping the pre-evaluation step.")
else:
    _ = evaluate_model(config)
    print("STEP 3 completed: Predictions obtained")

## Step 4: Retrieve the Thetas

Using the provided function `retrieve_thetas()`, we extract the latent ability parameters (thetas) that are used later in performance estimation.

In [ ]:
# Retrieve the thetas
thetas = retrieve_thetas(config)
print("STEP 4 completed: Thetas obtained")

## Step 5: Extract Samples and Define Test Set

We use the anchor indices to form a sampled dataset (for evaluation during search) and create a test set by dropping these anchor points from the full dataset.

In [ ]:
# Create sampled and test sets
sampled_df = df.loc[anchors].copy()
test_df = df.drop(anchors)

print("STEP 5 completed: Samples extracted")
print(f"Sampled data shape: {sampled_df.shape}")
print(f"Test data shape: {test_df.shape}")

## Step 6: Set Performance Estimation Parameters

We unpack some parameters from the configuration and create an instance of `ConfigPE` (performance estimation configuration) with the retrieved thetas, anchor weights, and other parameters.

In [ ]:
# Unpack parameters and define performance estimation configuration
pop_size = config.pop_size
n_iter = config.n_iter
run_id = config.run_id
bench = config.bench
mode = config.mode

est_parameters = ConfigPE(
    thetas=thetas,
    weights=anchors_weights,
    sample_ids=anchors,
    bench=bench,
    mode=mode,
    correct_metric=config.correctness_metric,
)

print("STEP 6 completed: Performance estimation parameters set")

## Step 7: Define the Merger

We now create a merger instance. In this example we use the `SlerpMerger`. The merger is initialized with the base model and one finetuned model (from the list in `ft_model_paths`), as well as other parameters such as the layer ranges and where to store the YAML configuration and merged model.

In [ ]:
# Define the merger
merger = SlerpMerger(
    run_id=run_id,
    path_to_base_model=config.ft_model_paths[0],
    path_to_model_1=config.ft_model_paths[1],
    path_to_store_yaml=config.path_to_store_yaml,
    path_to_store_merged_model=config.path_to_store_merged_model,
    dtype=config.dtype,
    layer_range_base_model=[0, 32],
    layer_range_model_1=[0, 32],
)

device = config.device if config.device else "cuda"
print("STEP 7 completed: Merger defined")

## Step 8: Define the Optimization Problem

We wrap the merging operation in an optimization problem. Here, `CrossLingualMathProblem` is used, which takes the merger, the test dataset, the sampled data, and additional configuration parameters such as the language id, performance estimation parameters, device, number of variables, objectives, and evaluation batch size.

In [ ]:
# Define the problem
problem = CrossLingualMathProblem(
    merger,
    test_df=test_df,
    search_df=sampled_df,
    lang_id=config.lang_id,
    conf_pe=est_parameters,
    device=device,
    n_var=11,
    n_obj=1,
    n_eq_constr=0,
    n_ieq_constr=0,
    discrete=True,
    eval_batch_size=config.eval_batch_size,
)

print("STEP 8 completed: Problem defined")

## Step 9: Define the Evolutionary Algorithm

We define a genetic algorithm (GA) for our optimization using a random integer sampling method, SBX crossover, and polynomial mutation. The algorithm is configured with a specified population size and duplicate elimination.

In [ ]:
# Define the evolutionary algorithm
algorithm = GA(
    pop_size=pop_size,
    sampling=IntegerRandomSampling(),
    crossover=SBX(),
    mutation=PM(),
    eliminate_duplicates=True,
)

print("STEP 9 completed: Algorithm defined")

## Step 10: Run the Evolutionary Search

Finally, we create a `Searcher` object with the problem, algorithm, and results path. Then we run the search process and test the best merged model found.

This step carries out the evolutionary search over the specified number of iterations.

In [ ]:
# Define the searcher and run the evolutionary search
searcher = Searcher(
    problem,
    algorithm,
    config.path_to_store_merged_model,
    n_iter,
    run_id=run_id,
    seed=config.seed,
    verbose=False,
)
searcher.search()
searcher.test()

print("STEP 10 completed: Search finished and testing done")

## Conclusion

In this notebook, we walked through the complete process of setting up and executing an evolutionary merging experiment using Mergenetic. 

You can now modify the configuration, change the merging method or optimization parameters, and run your own experiments.

**Happy merging!**